In [ ]:
import os
import sys
import time
import subprocess
import numpy as np

In [ ]:
def getGarfieldPath():
    """
    """
    filename = '../GARFIELDPATH'
    try:
        with open(filename, 'r') as file:
            garfieldPath = file.read().strip()
            if not os.path.exists(garfieldPath):
                print(f"Error: File 'setupGarfield.sh' not found at {garfieldPath}.")
                return None

    except FileNotFoundError:
        with open(filename, "w") as file:
            file.write('<<< Enter Garfield source path here. >>>')
            print(f"File '{filename}' created. Please update.")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None
        
    return garfieldPath

In [ ]:
def setupSimulation():
    """
    """

    #Check for log/
    if not os.path.exists("log"):
        os.makedirs("log")
        
    #Check for build/
    if not os.path.exists("build"):
        os.makedirs("build")
    
    #Make executable
    GARFIELDPATH = getGarfieldPath()
    makeBuild = (
        f'source {GARFIELDPATH} && '
        f'cd build && '
        f'cmake .. && '
        f'make'
    )
    result = subprocess.run(
        makeBuild,
        shell=True,
        check=True,
        executable='/bin/bash',
        capture_output=True,
        text=True
    )

    #Check for run number file
    if not os.path.exists('runNo'):
        with open('runNo', 'w') as file:
            file.write('1')
            
    return True

In [ ]:
def getRunNumber():
    """
    """
    filename = 'runNo'

    try:
        with open(filename, 'r') as file:
            content = file.read().strip()
            runNo = int(content)
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return -1
    except ValueError:
        print(f"Error: Invalid number format in '{filename}")
        return -1
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return -1
    
    return runNo

In [ ]:
def runGmsh():
    """
    """
    try:
        geoFile = 'GridPix.txt'
        with open(os.path.join(os.getcwd(), 'log/logGmsh.txt'), 'w+') as gmshOutput:
            startTime = time.monotonic()
            runReturn = subprocess.run(
                ['gmsh', os.path.join('./Geometry/', geoFile),
                 '-order', '2', '-optimize_ho',
                 '-clextend', '1',
                 '-setnumber', 'Mesh.OptimizeNetgen', '1',
                 '-setnumber', 'Mesh.MeshSizeFromPoints', '1',
                 '-3',
                 '-format', 'msh2'],
                stdout=gmshOutput, 
                check=True
            )
            endTime = time.monotonic()
            gmshOutput.write(f'\n\nGmsh run time: {endTime - startTime} s')

            if runReturn.returncode != 0:
                print('Gmsh failed. Check log for details.')
                return False
                
    except FileNotFoundError:
        print("Unable to write to 'log/logGmsh.txt'.")
        return False
        
    return True

In [ ]:
def runElmer():
    """
    """
    originalCWD = os.getcwd()
    os.chdir('./Geometry')

    os.makedirs("elmerResults", exist_ok=True)
        
    try:
        with open(os.path.join(originalCWD, 'log/logElmerGrid.txt'), 'w+') as elmerOutput:
            startTime = time.monotonic()
            runReturn = subprocess.run(
                ['ElmerGrid', '14', '2', 'GridPix.msh', 
                 '-names',
                 '-out', 'elmerResults', 
                 '-autoclean'], 
                stdout=elmerOutput,
                check=True
            )
            endTime = time.monotonic()
            elmerOutput.write(f'\n\nElmerGrid run time: {endTime - startTime} s')
            
            if runReturn.returncode != 0:
                print('ElmerGrid failed. Check log for details.')
                return False
            
        with open(os.path.join(originalCWD, 'log/logElmerSolver.txt'), 'w+') as elmerOutput:
            startTime = time.monotonic()
            runReturn = subprocess.run(
                ['ElmerSolver', 'GridPix.sif'],
                stdout=elmerOutput,
                check=True
            )
            endTime = time.monotonic()
            elmerOutput.write(f'\n\nElmerSolver run time: {endTime - startTime} s')

        if runReturn.returncode != 0:
                print('ElmerSolver failed. Check log for details.')
                return False
    finally:
        os.chdir(originalCWD)
    return True

In [ ]:
def runGarfield():
    """
    """
    originalCWD = os.getcwd()
    GARFIELDPATH = getGarfieldPath()
    os.chdir('./build/')
    try:
        with open(os.path.join(originalCWD, 'log/logGarfieldAvalanche.txt'), 'w+') as garfieldOutput:
            startTime = time.monotonic()
            setupAvalanche = (
                f'source {GARFIELDPATH} && '
                f'make && '
                f'./runAvalanche'
            )
            runReturn = subprocess.run(
                setupAvalanche, 
                stdout=garfieldOutput, 
                shell=True, 
                check=True
            )
            endTime = time.monotonic()
            garfieldOutput.write(f'\n\nGarfield run time: {endTime - startTime} s')

        if runReturn.returncode != 0:
                print('Garfield++ execution failed. Check log for details.')
                return False
    finally:
        os.chdir(originalCWD)
    return True

In [ ]:
def runSimulation():
    """
    """
    setupSimulation()
    
    runNo = getRunNumber()
    if runNo == -1:
        print("Error reading 'runNo'")
        return -1
    print(f'Running simulation - Run number: {runNo}')

    if not runGmsh():
        print('Error executing Gmsh.')
        return -1

    if not runElmer():
        print('Error executing Elmer.')
        return -1    

    if not runGarfield():
        print('Error executing Garfield.')
        return -1

    return runNo

In [ ]:
def writeSIF(gridVoltage):
    """
    """
    filename = os.path.join('./Geometry', 'GridPix.sif')
    try:
        with open(filename, 'r') as file:
            sifLines = file.readlines()  # Read all lines of the file
            
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return False 
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return False

    writeCathode = -1
    writeGrid = -1
    
    # Find the cathode and grid naming lines
    for i, line in enumerate(sifLines):
        if 'Name = "Cathode"' in line:
            writeCathode = i+1
        if 'Name = "Grid"' in line:
            writeGrid = i+1

    if writeCathode == -1 or 'Potential =' not in sifLines[writeCathode]:
        print('Error with cathode.')
        return False
    if writeGrid == -1 or 'Potential =' not in sifLines[writeGrid]:
        print('Error with grid.')
        return False

    #rewrite appropriate lines
    sifLines[writeCathode] = f"\tPotential = -{gridVoltage+20}\n"
    sifLines[writeGrid] = f"\tPotential = -{gridVoltage}\n"

    #write sif file
    try:
        with open(filename, 'w') as file:
            file.writelines(sifLines)
            
    except Exception as e:
        print(f"An error occurred while writing to {filename}: {e}")
        return False

In [ ]:
startTime = time.monotonic()
testRun = runSimulation()
endTime = time.monotonic()
print(f'Run {testRun} in {endTime - startTime}s)

In [ ]:
gridVoltages = np.linspace(380, 500, 13)
print(gridVoltages)

In [ ]:
allRunNos = []
for inVoltage in gridVoltages:
    writeSIF(inVoltage)
    inRun = runSimulation()
    allRunNos.append(inRun)

In [ ]:
print(allRunNos)

In [ ]:
newVoltage = [470, 480, 490, 500]
newRunNos = []
for inVoltage in newVoltage:
    writeSIF(inVoltage)
    inRun = runSimulation()
    newRunNos.append(inRun)
print(newRunNos)
